# 🧬 Clustering de cellules cancéreuses
Nous allons utiliser des méthodes de réduction de dimension (PCA) et de clustering (k-means) afin de regrouper des cellules cancéreuses (issues de sous-types de cancer distincts) en fonction de leur expression génique. 

# 💾 Données

> L'ensemble de donnée (dataset) que nous allons utiliser contient les valeurs d'expression génique de cellules cancéreuses. Ces données sont extraites d'un manuscrit rédigé par les chercheurs du projet d'analyse pan-cancéreuse The Cancer Genome Atlas (TCGA).
 
> Il y a 881 échantillons (lignes) représentant 5 sous-types de cancer distincts. Chaque échantillon comporte les valeurs d'expression génique pour 20 531 gènes (colonnes). L'ensemble de données est disponible sur le UC Irvine Machine Learning Repository.

# 📤 Import des Librairies

In [ ]:
import tarfile
import urllib

import pandas as pd # Analyse et manipulation de données (utilisation des Panda DataFrame) https://pandas.pydata.org/docs/user_guide/index.html and/or https://sparkbyexamples.com/python-pandas-tutorial-for-beginners/ 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns # Visualization de données
#import hvplot.pandas # Plots interactifs https://hvplot.holoviz.org/

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

# Téléchargement et extraction du dataset

Vous pouvez téléchager manuellement les fichiers de données depuis: https://archive.ics.uci.edu/dataset/401/gene+expression+cancer+rna+seq 

L'archive téléchargée devra alors être dé-zippée et enregistrée dans le dossier courrant: **UNIGE_DSMLAI_2025_11/Day_1_TP_2_Clustering**

In [ ]:
uci_tcga_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00401/"
archive_name = "TCGA-PANCAN-HiSeq-801x20531.tar.gz"

# URL
full_download_url = urllib.parse.urljoin(uci_tcga_url, archive_name)

# Télécharger le fichier
# r = urllib.request.urlretrieve (full_download_url, archive_name)

# Extraire les données de l'archive
tar = tarfile.open(archive_name, "r:gz")
tar.extractall()
tar.close()

In [ ]:
datafile = "TCGA-PANCAN-HiSeq-801x20531/data.csv"
labels_file = "TCGA-PANCAN-HiSeq-801x20531/labels.csv"

# Charger les données du fichier 'datafile'
data = np.genfromtxt(
    datafile,
    delimiter=",",
    usecols=range(1, 20532),
    skip_header=1
)

# Charger les étiquettes du fichier 'labels_file'
true_label_names = np.genfromtxt(
    labels_file,
    delimiter=",",
    usecols=(1,),
    skip_header=1,
    dtype="str"
)

Vérifions la taille des données; i.e.,
- Nombre d'échantillons (Nombre de lignes)
- Nombre de gènes (Nombre de colonnes)

In [ ]:
data.shape

**Instruction**: Vérifiez que le nombre d'étiquettes est consitent avec le nombre d'échantillons

<details>
<summary>Aide</summary>

1) <a href="https://numpy.org/doc/stable/reference/generated/numpy.shape.html">Utilisez numpy.shape</a>

2) Les étiquettes sont dans `true_label_names`

</details>

In [ ]:
# Code ici


Consultons les premières lignes et colonnes des données

In [ ]:
data[:5,:3]

**instruction** Consultez les étiquettes des 5 premiers échantillons

<details>
<summary>Aide</summary>
Ce sont les 5 premiers éléments de `true_label_names`
</details>


In [ ]:
# Code ici


In [ ]:
# Valeurs uniques dans le tableau des étiquettes de référence
np.unique(true_label_names)

# Convertir les abréviations en entiers avec LabelEncoder

In [ ]:
# Encode les étiquettes avec des valeurs entre 0 et le nombre de classes
label_encoder = LabelEncoder()

true_labels = label_encoder.fit_transform(true_label_names)
true_labels[:5]

In [ ]:
# Étant donné que l'encodeur a été ajusté aux données, vous pouvez voir les classes uniques représentées en utilisant .classes_.label_encoder.classes_
label_encoder.classes_

**Instruction**: Fixer le nombre de clusters aux nombres de classes uniques représentées
<details>
<summary>Aide</summary>
Vous pouvez utiliser la fonction 
<a href="https://www.w3schools.com/python/ref_func_len.asp">`len(...)`</a>
 
</details>

In [ ]:
# Code ici
n_clusters = None

#  Pipeline
## Pipeline pour traiter les données

**Instruction**: Fixez le nombre de composantes principales à 2 pour la méthode PCA
<details>
<summary>Aide</summary>
Indiquez l'argument `n_components`

Voir <a href="https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html">Documentation PCA</a>
</details>

In [ ]:
# Normalisation et réduction de dimension
preprocessor = Pipeline(
    [
        ("scaler", MinMaxScaler()),
        # Code ici
        ("pca", PCA(None,random_state=42))
    ]
)

## Pipeline pour effectuer le clustering K-means

In [ ]:
clusterer = Pipeline(
   [
       (
           "kmeans",
           KMeans(
               n_clusters=n_clusters,
               init="k-means++",
               n_init=50,
               max_iter=500,
               random_state=42,
           ),
       ),
   ]
)

## Enchaîner les pipelines de prétraitement et de clustering

In [ ]:
pipe = Pipeline(
    [
        ("preprocessor", preprocessor),
        ("clusterer", clusterer)
    ]
)

## Appliquer les étapes du pipeline sur les données

In [ ]:
pipe.fit(data)

In [ ]:
# Vérifie le résultat de la réduction de dimension (Transformation en 2D)
preprocessed_data = pipe["preprocessor"].transform(data)
preprocessed_data[:5,:]


In [ ]:
# Vérifie le résultat du clustering
predicted_labels = pipe["clusterer"]["kmeans"].labels_
print(predicted_labels.shape)
print(predicted_labels[:5])

# Évaluer la qualité du clustering

**Instruction**: Calculez le coefficient de silhouette en fonction des données réduites `preprocessed_data` et des clusters attribués `predicted_labels` 
<details>
<summary>Aide</summary>
Utilisez la fonction 
<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.silhouette_score.html">`silhouette_score(...)`</a>
</details>


In [ ]:
# Code ici


Profitons d'avoir les étiquettes de référence pour calculer une deuxième mesure de qualité du clustering :) 

**Instruction**: Calculez l'Indice de Rand Ajusté (ARI) en fonction des étiquettes de référence `true_labels` et des clusters attribués `predicted_labels` 
<details>
<summary>Aide</summary>
Utilisez la fonction 
<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html">`adjusted_rand_score(...)`</a>
</details>

In [ ]:
# Code ici


# Visualiser les données dans le contexte des étiquettes réelles et des étiquettes prédites

In [ ]:
# Construit le dataframe avec la projection des données en 2D, leurs clusters prédits, et leurs étiquettes de référence
pcadf = pd.DataFrame(
    pipe["preprocessor"].transform(data)[:,:2],
    columns=["component_1", "component_2"],
)

pcadf["predicted_cluster"] = pipe["clusterer"]["kmeans"].labels_
pcadf["true_label"] = label_encoder.inverse_transform(true_labels)
pcadf.head()

In [ ]:
# Plot
plt.style.use("fivethirtyeight")
plt.figure(figsize=(8, 8))

scat = sns.scatterplot(
    data=pcadf,
    x = "component_1",
    y = "component_2",
    s=50,
    hue="predicted_cluster",
    style="true_label",
    palette="Set2",
)

**QUESTION:** 
- Comparez les clusters/étiquettes prédites (couleurs) et les étiquettes réelles (formes).

# Réglage des paramètres
Et si nous ne connaissions pas les étiquettes réelles des données...

1) __Comment aurions-nous pu "trouver" le nombre de clusters (2, 5 ou 20 types de cellules)?__

2) __Pourquoi baser le clustering sur 2 dimensions seulement? Pourrions-nous obtenir de meilleurs résultats avec davantage de dimensions/composantes?__

## Réduction de dimension
Utilisez les la variance expliquée cumulée pour identifier le nombre approprié de composantes principales à garder.

In [ ]:
# Récupére et visualise la variance expliquée cumulée par les 20 premières composantes
m = 20
pipe["preprocessor"]["pca"].n_components = m
pipe.fit(data)
pipe["preprocessor"]["pca"].explained_variance_
plt.plot(
    range(1, m+1),
    np.cumsum(pipe["preprocessor"]["pca"].explained_variance_ratio_))

**QUESTION:**

A partir d''un certain nombre de composantes principales, le gain en terme de variance expliquée devient moindre.

- A combien estimez-vous ce nombre (de composantes principales)?

N'hésitez pas à tester dans la suite du code différentes valeurs pour le nombre de composantes principales à garder/utiliser et évaluer l'impact sur le clustering.

## Clustering
Enfin, utilisons les métriques de qualité de clustering pour définir le nombre clusters

**Instruction**: 
- Fixez le nombre de composantes principales `n_components` à 7
- Faites varier le nombre de clusters `n_clusters` de 2 à 10
<details>
<summary>Aide</summary>

- Vous pouvez accéder aux hyperparamètre de la PCA, à travers `pipe["preprocessor"]["pca"]`

- Vous pouvez accéder aux hyperparamètre de K-means clustering, à travers `pipe["clusterer"]["kmeans"]`
</details>

In [ ]:
# Listes vides pour stocker les métriques de qualité 
# Silhouette
silhouette_scores = []
# Index Rand Ajusté
ari_scores = []
for n in range(2, 11):
    # Faites varier le nombre de clusters,
    # et laissez le nombre de composantes principales fixé à 7
    ### Code ici - Début ###
    
    
    ### Code ici - Fin ###
    pipe.fit(data)

    silhouette_coef = silhouette_score(
        pipe["preprocessor"].transform(data),
        pipe["clusterer"]["kmeans"].labels_,
    )
    ari = adjusted_rand_score(
        true_labels,
        pipe["clusterer"]["kmeans"].labels_,
    )

    # Ajoute les métriques aux listes correspondantes
    silhouette_scores.append(silhouette_coef)
    ari_scores.append(ari)

# Visualisez la relation entre le nombre de clusters et la qualité du clustering

In [ ]:
plt.style.use("fivethirtyeight")
plt.figure(figsize=(6, 6))
plt.plot(
    range(2, 11),
    silhouette_scores,
    c="#008fd5",
    label="Silhouette Coefficient",
)
plt.plot(range(2, 11), ari_scores, c="#fc4f30", label="ARI")

plt.xlabel("n_clusters")
plt.legend()
plt.title("Clustering Performance as a Function of n_clusters")
plt.tight_layout()
plt.show()


# 📝 Résumé
Dans ce notebook, vous avez utilisé des méthodes de dimension de réduction et de clustering pour le regroupement automatique de cellules cancéreuses.

Vous avez couvert les aspects suivants:
> - Le modèle de réduction de dimension PCA
> - Le modèle de clustering K-means
> - L'exploration et la préparation des données avant leur utilisation pour la réduction de dimension et le clustering
> - L'entrainement des deux modèles PCA et K-means
> - L'exploration des hyperparamètres des deux modèles, i.e., nombre de composantes principales, et nombre de clusters
> - L'évaluation de la qualité du clustering à l'aide du coefficient Silhouette et Score de Rand Ajusté 


# 🔗 References:
- [Scikit-learn library - Clustering](https://scikit-learn.org/stable/modules/clustering.html)
- [K-Means Clustering in Python: A Practical Guide](https://realpython.com/k-means-clustering-python/)
- [How to Select the Best Number of Principal Components for the Dataset](https://towardsdatascience.com/how-to-select-the-best-number-of-principal-components-for-the-dataset-287e64b14c6d)